In [10]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

In [16]:
class SimpleDataGenerator:
    def __init__(self, subscription_id, resource_group, workspace_name):
        """Initialize Azure ML client."""
        self.ml_client = MLClient(
            DefaultAzureCredential(),
            #subscription_id=subscription_id,
            #resource_group=resource_group,
            #workspace_name=workspace_name
        )

    def generate_data(self, n_samples=1000, output_file="energy_data.csv"):
        """Generate synthetic energy requirement data."""
        # Generate synthetic data
        X, y = make_classification(
            n_samples=n_samples,
            n_features=10,
            n_informative=5,
            n_redundant=2,
            n_classes=2,
            random_state=42
        )

        # Create feature names and DataFrame
        feature_names = [
            "Temperature (°C)", "Humidity (%)", "Energy Usage (kWh)",
            "Peak Hours (hours)", "Cost of Energy ($)", "Weather Index",
            "Appliance Load (%)", "Solar Output (kWh)", "Wind Output (kWh)", "Seasonality Index"
        ]
        df = pd.DataFrame(X, columns=feature_names)
        df["Energy_Requirement"] = np.where(y == 1, "Yes", "No")

        # Save data locally
        df.to_csv(output_file, index=False)
        print(f"Data generated and saved locally at: {output_file}")
        return output_file

    def upload_to_datastore(self, local_file, datastore_path):
        """Upload the generated data to Azure ML datastore."""
        datastore = self.ml_client.datastores.get("workspaceblobstore")  # Default datastore
        datastore.upload(
            local_path=local_file,
            target_path=datastore_path,
            overwrite=True
        )
        print(f"Data uploaded to datastore at: {datastore_path}")

In [18]:
if __name__ == "__main__":
    # Azure ML Workspace details
    subscription_id="9c15fe3b-278f-4875-b39a-d756406c9775"
    resource_group="DP-100"
    workspace_name="dp_100_learning"

    # Initialize the data generator
    data_generator = SimpleDataGenerator(subscription_id, resource_group, workspace_name)

    # Generate synthetic data
    local_file = data_generator.generate_data(n_samples=1000, output_file="../data/energy_data/input_data/energy_data.csv")

    # Upload the data to Azure ML datastore
    #data_generator.upload_to_datastore(local_file, datastore_path="datasets/energy_data/")

Data generated and saved locally at: ../data/energy_data/input_data/energy_data.csv
